# ET0 Calculation
In this notebook, the reference evapotranspiration (ET0) calculation process is summarized. The computation is done in the script **process_data.py**.

### Notes
* Equations and the accompanying clarifications are taken from the ["FAO Irrigation and drainage paper 56"](http://www.fao.org/3/x0490e/x0490e00.htm#Contents).
* The functions used in this notebook are found in *ETProject/ET0_functions.py*

ET0 equation:

$$ ET_0 = \frac{0.408 \Delta \cdot \left( R_n - G \right) + \gamma \cdot \frac{900}{T + 273} \cdot u_2 \cdot \left(e_s - e_a \right)}{\Delta + \gamma \cdot \left( 1 + 0.34 \cdot u_2 \right)} $$

Where:

* $ET_0$ reference evapotranspiration ($mm \cdot day^{-1}$)
* $R_n$ net radiation ($MJ \cdot m^{-2} \cdot day^{-1}$)
* $G$ soil heat flux density ($MJ \cdot m^{-2} \cdot day^{-1}$)
* $T$ mean daily air temperature at 2 m height (°C) **calculated as the mean between max and min temperatures**
* $u_2$ wind speed at 2 m height ($m \cdot s^{-1}$)
* $e_s$ saturation vapour pressure ($kPa$)
* $e_a$ actual vapour pressure ($kPa$)
* $\Delta$ slope vapour pressure deficit ($kPa \cdot °C^{-1}$)
* $\gamma$ psychometric constant ($kPa \cdot °C^{-1}$)

#### Required variables:

1. Elevation ($m$)
2. Latitude (degrees)
3. Max and min Temperatures $T_{min}$, $T_{max}$ ($°C$)
4. Max and min relative humidities $RH_{min}$, $RH_{max}$ (%)
5. Day of year
6. Average wind speed $u_z$ ($m \cdot s^{-1}$)
7. Solar Radiation ($MJ \cdot m^{-2} \cdot day^{-1}$) or sunshine duration (hr)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Changing default directory to the project's main directory
os.chdir(Path.cwd().parent)

# Loading ET0 function definitions from ETProject
from ETProject.ET0_functions import *

### Atmospheric Pressure (P)

$$ P = 101.3 \cdot \left( \frac{293-0.0065 \cdot z}{293} \right) ^{5.26} $$

### Latent heat of vaporization ($\lambda$)
The latent heat of vaporization expresses the energy required to change a unit mass of water from liquid to water vapour in a constant pressure and constant temperature process.

$$ \lambda = 2.45 \space MJ \cdot kg^{-1} $$

This is the latent heat for an air temperature of about 20°C, however it is accepted for normal temperature variations in the FAO 56 PM equation.

### Psychrometric Constant ($\gamma$)
$$ \gamma = \frac{c_p \cdot P}{\epsilon \cdot \lambda} = 0.665 \times 10^{-3} \cdot P$$
Where:
* $\gamma$ psychrometric constant ($ kPa \cdot °C^{-1} $)
* P atmospheric pressure ($kPa$)
* $\lambda$ latent heat of vaporization ($MJ \cdot kg^{-1}$)
* $c_p$ specific heat at constant pressure
    * $ c_p = 1.013 \cdot 10^{-3} MJ \cdot kg^{-1} °C^{-1} $
* $\epsilon$ ratio molecular weight of water vapour/dry air
    * $ \epsilon = 0.622 $

In [4]:
psych_constant?

In [2]:
z = 1800
gam = psych_constant(z)

print('Psychrometric Constant = {:.3f}'.format(gam))

Psychrometric Constant = 0.054


### Air Temperature
"The daily maximum air temperature (Tmax) and daily minimum air temperature (Tmin) are, respectively, the maximum and minimum air temperature observed during the 24-hour period, beginning at midnight. *Tmax and Tmin for longer periods such as weeks, 10-days or months are obtained by dividing the sum of the respective daily values by the number of days in the period.*"

$$ T_{mean} = \frac{T_{max} + T_{min}}{2} $$

**In the source the equation has a - sign instead of a + sign**

### Air Humidity
"Relative humidity and dewpoint temperature data are notoriously plagued by measurement errors. Measurement error is common for both older hygrothermograph types of instruments and for the more modem electronic instruments. Frequently, it is better to utilize a dewpoint temperature that is predicted from daily minimum air temperature, rather than to use unreliable relative humidity measurements. The user is encouraged to utilize good judgement in this area."

### Mean Saturation Vapour Pressure ($e_s$)
* Saturation vapour pressure:
$$ e°(T) = 0.6108 \cdot exp \left( \frac{17.27 \cdot T}{T + 237.3} \right) $$
Where:
    * $e°(T)$ saturation vapour pressure at the air temperature T (kPa)
    * T air temperature (°C)
    

* Mean saturation vapour pressure:
$$ e_s = \frac{e°(T_{max}) + e°(T_{min})}{2} $$

#### Notes:
* Due to the non-linearity of the above equation, the mean saturation vapour pressure for a day, week, decade or month should be computed as the mean between the saturation vapour pressure at the mean daily maximum and minimum air temperatures for that period.
* Using mean air temperature instead of daily minimum and maximum temperatures results in lower estimates for the mean saturation vapour pressure.

In [5]:
sat_vap_pressure?

In [3]:
T1 = 24.5
T2 = 15
Tmean = (T1 + T2) / 2
es_maxmin = (sat_vap_pressure(T1) + sat_vap_pressure(T2)) / 2
es_mean = sat_vap_pressure(Tmean)
print('es max-min = {:.3f}'.format(es_maxmin))
print('es mean = {:.3f}'.format(es_mean))

es max-min = 2.390
es mean = 2.302


### Slope of Saturation Vapour Pressure Curve ($\Delta$)
$$ \Delta = \frac{4098 \cdot \left( 0.6108 \cdot exp \left(\frac{17.27 \cdot T}{T + 237.3} \right) \right)}{(T + 237.3)^2} $$

Where:
* $\Delta$ slope of saturation vapour pressure curve at air temperature T ($kPa \cdot °C^{-1}$)
* T mean air temperature calculated using max and min temperatures as explained earlier (°C)

### Actual vapour pressure ($e_a$) derived from relative humidity data

* Using $RH_{max}$ and $RH_{min}$:

$$ e_a = \frac{e°(T_{min}) \cdot \frac{RH_{max}}{100} + e°(T_{max}) \cdot \frac{RH_{min}}{100}}{2} $$

* When using equipment where errors in estimating $RH_{min}$ can be large, or when RH data integrity are in doubt, then one should use only $RH_{max}$:

$$ e_a = e°(T_{min}) \cdot \frac{RH_{max}}{100} $$

In [6]:
actual_vap_pressure?

In [4]:
Tmin = 18
Tmax = 25
RHmin = 54
RHmax = 82
print('maxmin = True:\n{:.3f}'.format(actual_vap_pressure(Tmin, RHmax, Tmax, RHmin, maxmin = True)))
print('maxmin = False:\n{:.3f}'.format(actual_vap_pressure(Tmin, RHmax, Tmax, RHmin, maxmin = False)))

maxmin = True:
1.702
maxmin = False:
1.692


### Radiation

#### Extraterrestrial Radiation $R_a$:
Extraterrestrial radiation is the total radiation that should theoretically reach the earth from the sun, excluding the radiation absorbed by the atmosphere. It is calculated using a constant value of sun radiation modified by the time and location of the studied are, which affects the angle of arrival of the solar radiation to the earth.

$R_a$ for a certain day and location:

$$ R_a = \frac{24 \cdot 60}{\pi} \cdot G_{sc} \cdot d_r \cdot \left[ \omega_s \cdot sin(\varphi) \cdot sin(\delta) + cos(\varphi) \cdot cos(\delta) \cdot sin(\omega_s) \right] $$

Where:
* $R_a$ extraterrestrial radiation $MJ \cdot m^{-2} \cdot day^{-1}$
* $G_{sc}$ solar constant ($0.082 MJ \cdot m^{-2} \cdot min^{-1}$)
* $d_r$ inverse relative distance Earth-Sun
* $\omega_s$ sunset hour angle (rad)
* $\varphi$ latitude (rad) - positive for northern hemisphere and negative for southern hemisphere
* $\delta$ solar decimation (rad)

$$ d_r = 1 + 0.033 \cdot cos \left( \frac{2\pi}{365} \cdot J \right) $$

$$ \delta = 0.409 \cdot sin \left( \frac{2\pi}{365} \cdot J - 1.39 \right) $$ 

Where:
* J number of the day in the year (1-365/366) (see Table 2.5 in Annex 2)

$$ \omega_s = arccos \left( -tan(\varphi) \cdot tan(\delta) \right) $$

In [7]:
extra_terr_rad?

In [5]:
# Example 8.
lat = -20
day = 246
Ra = extra_terr_rad(lat, day=day)
print('Ra = {:.2f}'.format(Ra))

Ra = 32.19


#### Solar or Shortwave Radiation $R_s$:
Solar radiation $R_s$ is the actual amount of radiation that reaches the earth from the sun in a certain point. This value could be measured or computed from sunshine hours using the Angstrom equation:

$$ R_s = (a_s + b_s \frac{n}{N})R_a $$

Where:
* $a_s$ Fraction of extraterrestrial radiation reaching the earth on overcast days (n = 0)
* $a_s + b_s$ Fraction of extraterrestrial radiation reaching the earth on clear days (n = N)
* $n$: Daily measured sunlight hours
* $N$: Total daily day hours (according to time and location)

$$ N = \frac{24}{\pi} \omega_s $$

#### Clear Sky Solar Radiation $R_{s0}$:
$R_{so}$ is the solar radiation that would reach the earth when the sky is clear. It is estimated from the extraterrestrial radiation $R_a$. $\frac{R_s}{R_{so}}$  is therefore the ratio of actual solar radiation to the clear sky solar radiation, ranging between 0.33 and 1.

$$ R_{so} = (0.75 + 2 \cdot 10^{-5} \cdot Z) \cdot R_a $$

Where:
* $R_{so}$ in $(MJ \cdot m^{-2} \cdot day^{-1})$
* $Z$ station elevation above sea level ($m$)

#### Albedo (α) and Net Solar Radiation ($R_{ns}$):
Albedo is the fraction of solar radiation reflected by the earth. It depends on the earth cover. For the reference crop used in ET0 estimation, α=0.23.

$$ R_{ns} = (1-\alpha) \cdot R_s$$

Where:
* $ R_{ns}$ in $(MJ \cdot m^{-2} \cdot day^{-1})$
* $\alpha$ albedo, 0.23 for the hypothetical grass reference crop
* $R_s$ incoming solar radiation $(MJ \cdot m^{-2} \cdot day^{-1} )$

#### Net Longwave Radiation ($R_{nl}$):
During the processes in which the earth is heated by radiation, the earth loses energy and some of this lost energy is constituted of emitted radiation. Some of this emitted radiation is absorbed by the atmosphere, heating it in the process and is returned eventually to the earth. The remaining part is lost in space. Net longwave radiation is the radiation lost from the earth in these processes. As the outgoing longwave radiation is almost always greater than me incoming longwave radiation, $R_{nl}$ represents an energy loss.

$$ R_{nl} = \sigma \cdot \left[ \frac{T_{max, K}^4 + T_{min, K}^4}{2} \right] \cdot \left(0.34 - 0.14 \sqrt{e_a} \right) \cdot \left( 1.35 \cdot \frac{R_s}{R_{so}} - 0.35 \right) $$

Where:
* $R_{nl}$ in $(MJ \cdot m^{-2} \cdot day^{-1})$
* $\sigma$ Stefan-Boltzmann constant ($ 4.903 \cdot 10^{-9} MJ \cdot K^{-4} \cdot m^{-2} \cdot day ^{-1} $)
* $T_{max, K}$, $T_{min, K}$ maximum and minimum absolute temperature during the 24-hour period ($K = °C + 273.16$)
* $e_a$ actual vapour pressure ($kPa$)
* $\frac{R_s}{R_{so}} \leq 1$ relative shortwave radiation
* $R_s$, $R_{so}$ in ($MJ \cdot m^{-2} \cdot day^{-1}$)

In [ ]:
clear_sky_rad?

In [ ]:
net_long_rad?

In [6]:
# Example 11
lat = -22.7
Tmax = 25.1
Tmin = 19.1
ea = 2.1
Rs = 14.5
elev = 2
Ra = 25.1

Rso = clear_sky_rad(elev, ex_rad = Ra)
Rnl = net_long_rad(Tmax, Tmin, ea, Rs, Rso)

print('Rso = {:.2f}'.format(Rso))
print('Rnl = {:.2f}'.format(Rnl))

Rso = 18.83
Rnl = 3.53


#### Net Radiation ($R_n$):
It is the difference between the incoming net shortwave $R_{ns}$, and the net outgoing longwave $R_{nl}$ radiation. It is normally positive in daytime and negative at night. In a 24-hour period $R_n$ is usually positive.

$$ R_n = R_{ns} - R_{nl} $$

#### Soil Heat Flux (G):
The soil heat flux, G, is the energy that is utilized in heating the soil. G is positive when the soil is warming and negative when the soil is cooling.

$$ G_{month, i} = 0.07 \cdot \left( T_{month, i+1} - T_{month, i-1} \right) $$

Or:

$$ G_{month, i} = 0.14 \cdot \left( T_{month, i} - T_{month, i-1} \right) $$

Where:
* $T_{month, i}$ mean air temperature of month i (°C)



#### Note:
The standard energy unit used by FAO in the FAO-56 PM handbook is $MJ \cdot m^{-2} \cdot day^{-1} $. This is then converted to equivalent evaporation in $mm \cdot day^{-1}$ using the following function:

$$ R_{\text{depth of water}} = \frac{R_{\text{energy/surface}}}{\lambda \cdot \rho_w} \space\space (m \cdot day^{-1})$$

Where:
* $\lambda$ latent heat of vaporization ($2.45 \space MJ \cdot kg^{-1}$)
* $\rho_w$ density of water ($1000 \space kg \cdot m^{-3}$)

By substituting in the values we find the radiation in $mm \cdot day^{-1}$ is:

$$ R_{mm\cdot day^{-1}} = \frac{R_{MJ \cdot m^{-2} \cdot day^{-1}}}{2.45} = 0.408 \cdot R_{MJ \cdot m^{-2} \cdot day^{-1}} $$

### Wind Speed
According to the [**Turkish State Meteorological Service**](https://mgm.gov.tr/genel/meteorolojikaletler.aspx?s=9), wind speed is measured at a height of 10m above the ground in weather stations.

To calculate the equivalent wind speed at a height 2m above the ground:

$$ u_2 = u_z \cdot \frac{4.87}{ln \left( 67.8 Z - 5.42 \right)} $$

Where:
* $u_2$ wind speed at 2m above the ground ($m \cdot s^{-1}$)
* $u_z$ wind speed at z m above the ground ($m \cdot s^{-1}$)
* Z height of measurement above the ground ($m$)

In [ ]:
windspeed_2m?

In [7]:
# Example 14.
uz = 3.2
z = 10
print('U2 = {:.2f}'.format(windspeed_2m(z, uz)))

U2 = 2.39


### Reference Evapotranspiration (ET0)

ET0 equation:

$$ ET_0 = \frac{0.408 \Delta \cdot \left( R_n - G \right) + \gamma \cdot \frac{900}{T + 273} \cdot u_2 \cdot \left(e_s - e_a \right)}{\Delta + \gamma \cdot \left( 1 + 0.34 \cdot u_2 \right)} $$

#### FAO paper 56 - Chapter 4 - Example 17

In [8]:
compute_ET0?

In [8]:
lat = 13.73
elev = 2
month = 4
max_temp = 34.8
min_temp = 25.6
avg_temp = 30.2
max_hum = 86.817
min_hum = 0
avg_ws = 2
inc_rad = 22.65
G = 0.14

compute_ET0(latitude=lat, elev=elev, month=month, max_temp=max_temp,
                min_temp=min_temp, avg_temp=avg_temp, RHmax=max_hum,
                RHmin= min_hum, avg_ws=avg_ws,
               inc_rad=inc_rad, G=G, maxmin=False, G_method=0, compute_inc_rad=False)

{'psych_const': 0.06734878003684622,
 'es': 4.421797805749032,
 'ea': 2.8500034464652364,
 'sat_slope': 0.2458002831073227,
 'Ra': 38.08763099465575,
 'Rso': 28.567246751231597,
 'Rns': 17.4405,
 'Rnl': 3.1047701797047287,
 'Rn': 14.33572982029527,
 'ET0': 5.7169772124324485}